In [14]:
import pandas as pd
import string
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
import torch
from torch.autograd import Variable

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27448 entries, 0 to 27447
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27447 non-null  object
 1   sentiment  27448 non-null  object
dtypes: object(2)
memory usage: 429.0+ KB


In [4]:
def text_processor():
    df['text'] = df['text'].apply(lambda x : remove_stopwords(str(x)))
    df['text'] = df['text'].apply(lambda x : str(x).lower().translate(str.maketrans('','',string.punctuation)))
    df['text'] = df['text'].apply(lambda x : ''.join([i for i in x if not i.isdigit()]))
    # df['text_list'] = df['text_list'].apply(lambda x : np.asarray(x))

text_processor()

In [5]:
df.head()

,text,sentiment
0,oh marly im sorry i hope soon,neutral
1,playing ghost online interesting the new updat...,positive
2,cleaning house family comming later today,neutral
3,gotta restart i thought win supposed end cons...,neutral
4,see wat i mean bout follw friidays its called ...,neutral


In [6]:
def tokenize_corpus(text):
    tokens = str(text).split()
    return tokens

df['tokenized_text'] = df['text'].apply(lambda x : tokenize_corpus(x))

In [7]:
df.head()

,text,sentiment,tokenized_text
0,oh marly im sorry i hope soon,neutral,"[oh, marly, im, sorry, i, hope, soon]"
1,playing ghost online interesting the new updat...,positive,"[playing, ghost, online, interesting, the, new..."
2,cleaning house family comming later today,neutral,"[cleaning, house, family, comming, later, today]"
3,gotta restart i thought win supposed end cons...,neutral,"[gotta, restart, i, thought, win, supposed, en..."
4,see wat i mean bout follw friidays its called ...,neutral,"[see, wat, i, mean, bout, follw, friidays, its..."


In [8]:
vocabulary = []

for sentence in df['tokenized_text']:
    for element in sentence:
        if element not in vocabulary:
            vocabulary.append(element)

In [10]:
word2idx = {w : idx for(idx, w) in enumerate(vocabulary)}
idx2word = {idx : w for(idx, w) in enumerate(vocabulary)}

In [11]:
vocabulary_size = len(vocabulary)

In [12]:
window_size = 2
idx_pairs = []

for sentence in df['tokenized_text']:
    indices = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(indices)):
        for w in range(-window_size, window_size+1):
            context_word_pos = center_word_pos + w
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))
            
idx_pairs - np.array(idx_pairs)

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [16]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [19]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad =True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)

num_epochs = 100
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        
        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
        
        log_softmax = torch.nn.functional.log_softmax(z2, dim=0)
        
        loss = torch.nn.functional.nll_loss(log_softmax.view(1, -1), y_true)
        loss_val += loss.data
        loss.backward()

        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()

    if epo % 10 == 0:
        print(f'Loss at epo {epo} : {loss_val/len(idx_pairs)}')

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number